#### Business Analytics FHDW 2025
## Aufgabe

Wir betrachten den bekannten Datensatz *BostonHousing.csv* über Stadtgebiete (*tracts*) im Rahmen einer Volkszählung. Die Daten weisen ein ethisches Problem auf (Kategorie *b*, "the proportion of blacks by town", im Jahre 1978 offenbar ein akzeptiertes Kriterium, heute in der Literatur oft einfach ausgelassen), das wir an dieser Stelle nur technisch lösen können.

Ermitteln Sie ein geeignetes k-NN-Vorhersagemodell für die Zielvariable *medv* (Medianwert von eigentümerbewohnten Häusern in $ 1.000):
* Berücksichtigen Sie alle 12 Prädiktoren.
* Prüfen Sie *k* von 1 bis 5. Was ist das beste *k*, was bedeutet der Wert?
* Wir haben es nun mit einer numerischen Zielvariable zu tun, Sie benötigen also den `KNeighborsRegressor` und auch einen geeigneten Wert (statt `accuracy_score`) zur Beurteilung der *k*s.
* Denken Sie an die Normierung der Daten.

Sagen Sie mit dem besten *k* den Zielwert für den neuen Datenpunkt `{'crim': 0.2, 'zn': 0, 'indus':7, 'chas':0, 'nox':0.538, 'rm': 6, 'age':62, 'dis':4.7, 'rad':4, 'tax':307, 'ptratio':21, 'lstat':10}` voraus.

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor

housing_df = pd.read_csv('./Daten/BostonHousing.csv')
del housing_df['b']
housing_df

In [ ]:
train_data, valid_data = train_test_split(housing_df, test_size=0.4, random_state=1)

relevant_cols = housing_df.drop(columns=['medv']).columns
print(relevant_cols)

# Umbenennung nicht zwingend für das Ergebnis, aber üblich und sauberer im Vorgehen:
relevant_cols_norm = ['z'+col for col in relevant_cols]
print(relevant_cols_norm)

scaler = preprocessing.StandardScaler()
# Hier darauf achten, dass die Werte der Zielvariable nicht normiert werden:
scaler.fit(train_data[relevant_cols])

In [ ]:
housing_norm = pd.concat([pd.DataFrame(scaler.transform(housing_df[relevant_cols]),
                                       columns=relevant_cols_norm),
                          housing_df[['medv']]], axis=1)

housing_norm

In [ ]:
train_norm = housing_norm.iloc[train_data.index]
valid_norm = housing_norm.iloc[valid_data.index]

train_X = train_norm[relevant_cols_norm]
train_y = train_norm['medv']
valid_X = valid_norm[relevant_cols_norm]
valid_y = valid_norm['medv']
results = []
for k in range(1, 6):
    knn = KNeighborsRegressor(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'RMSE': root_mean_squared_error(valid_y, knn.predict(valid_X))
    })
    
results = pd.DataFrame(results)
results

In [ ]:
# k = 3 weist den niedrigsten Fehler auf, also machen wir damit weiter:
knn = KNeighborsRegressor(n_neighbors=3).fit(housing_norm[relevant_cols_norm], housing_norm['medv'])
# Den neuen Datenpunkt legen wir uns an:
new_tract = pd.DataFrame([{'crim': 0.2, 'zn': 0, 'indus':7, 'chas':0, 'nox':0.538, 'rm': 6,
                          'age':62, 'dis':4.7, 'rad':4, 'tax':307, 'ptratio':21, 'lstat':10}])
# Auch den neuen Datenpunkt müssen wir normieren:
new_tract_norm = pd.DataFrame(scaler.transform(new_tract), columns=relevant_cols_norm)

# Ausführliches Ergebnis mit den drei Nachbarn: 
distances, indices = knn.kneighbors(new_tract_norm)
print(new_tract_norm)
print('Abstände der drei Nachbarn vom neuen Datenpunkt:', distances)

# Und die eigentliche Vorhersage:
print(knn.predict(new_tract_norm))
train_norm.iloc[indices[0], :]

Bedeutung des *k*: Aus den *medv*-Werten der *k* Nachbarn und den Distanzen errechnet sich ein Mittelwert als Prädiktion der Zielvariable.